# api global power plant

In [1]:
import pandas as pd
from pandas.io import sql
import sqlalchemy as db
from datetime import datetime, date
import numpy as np
from scipy import stats
import pymysql

In [2]:
global_power_plant = pd.read_csv('global_power_plant_database.csv')


C:\Users\Fernando Aleisy\AppData\Local\Temp\ipykernel_14808\2295192830.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  global_power_plant = pd.read_csv('global_power_plant_database.csv')


In [3]:
plantas_energia = global_power_plant[['country',
                                     'country_long',
                                     'name',
                                     'gppd_idnr',
                                     'capacity_mw',
                                     'latitude',
                                     'longitude',
                                     'primary_fuel',
                                     'other_fuel1',
                                     'other_fuel2',
                                     'other_fuel3',
                                     'source',
                                     'geolocation_source']]

In [4]:
plantas_energia['other_fuel1'].astype('str')
plantas_energia['other_fuel2'].astype('str')
plantas_energia['other_fuel3'].astype('str')

0        nan
1        nan
2        nan
3        nan
4        nan
        ... 
34931    nan
34932    nan
34933    nan
34934    nan
34935    nan
Name: other_fuel3, Length: 34936, dtype: object

In [5]:
plantas_energia["other_fuel"] = plantas_energia['other_fuel1'] + ' , ' + plantas_energia['other_fuel2'] + ' , ' + plantas_energia['other_fuel3']

C:\Users\Fernando Aleisy\AppData\Local\Temp\ipykernel_14808\1117253419.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plantas_energia["other_fuel"] = plantas_energia['other_fuel1'] + ' , ' + plantas_energia['other_fuel2'] + ' , ' + plantas_energia['other_fuel3']


In [6]:
plantas_energia = plantas_energia.drop('other_fuel1',axis=1)
plantas_energia = plantas_energia.drop('other_fuel2',axis=1)
plantas_energia = plantas_energia.drop('other_fuel3', axis=1)

In [7]:
plantas_energia.fillna(0,inplace=True)

In [8]:
plantas_energia = plantas_energia.reindex(columns=['country',
                                                   'country_long',
                                                   'name',
                                                   'primary_fuel',
                                                   'other_fuel',
                                                   'source',
                                                   'capacity_mw',
                                                   'gppd_idnr',
                                                   'latitude',
                                                   'longitude',
                                                   'geolocation_source'])

In [9]:
plantas_energia.columns = ['Codigo Pais',
                           'Pais',
                           'Nombre',
                           'Combustible Principal',
                           'Otros Combustibles',
                           'source',
                           'Capacidad MW',
                           'GPPD IDNR',
                           'Latitud',
                           'Longitud',
                           'geolocation_source']

In [10]:
equivalencias_paise = {'Bangladesh': 'Bangladés',
                        'Bahrein': 'Baréin',
                        'Belarús': 'Bielorrusia',
                        'Bhután': 'Bután',
                        'Botswana': 'Botsuana',
                        "Côte d'Ivoire": 'Costa de Marfil',
                        'Congo, República Democrática del': 'República Democrática del Congo',
                        'Congo, República del': 'República del Congo',
                        'Djibouti': 'Yibuti',
                        'Egipto, República Árabe de': 'Egipto',
                        'Fiji': 'Fiyi',
                        'Micronesia (Estados Federados de)': 'Micronesia',
                        'Guinea-Bissau': 'Guinea-Bisáu',
                        'Irán, República Islámica del': 'Irán',
                        'Iraq': 'Irak',
                        'Kazajstán': 'Kazajistán',
                        'Kenya': 'Kenia',
                        'Corea, República de': 'Corea del Sur',
                        'Corea, República Popular Democrática de': 'Corea del Norte', 
                        'República Democrática Popular Lao': 'Laos',
                        'Lesotho': 'Lesoto',
                        'Viet Nam': 'Vietnam',
                        'Yemen, Rep. del': 'Yemen',
                        'Zimbabwe': 'Zimbabue',
                        'Myanmar': 'Birmania',
                        'Nueva Zelandia': 'Nueva Zelanda',
                        'Palau': 'Palaos',
                        'Papua Nueva Guinea': 'Papúa Nueva Guinea',
                        'Suriname': 'Surinam',
                        'Eswatini': 'Suazilandia',
                        'República Árabe Siria': 'Siria',
                        'Timor-Leste': 'Timor Oriental'}

In [11]:
plantas_energia.replace({'Pais': equivalencias_paise}, inplace = True)

In [12]:
paises = pd.read_csv('Paises.csv')

paises.sort_values(by='Nombre_Pais', inplace=True)

plantas_energia = plantas_energia[plantas_energia['Pais'].isin(paises['Nombre_Pais'])]

In [13]:
print(f"La lista contiene {len(paises)} países, y el DataFrame {len(plantas_energia['Pais'].unique())} países")

La lista contiene 198 países, y el DataFrame 70 países


In [14]:
plantas_energia = plantas_energia.sort_values('Pais')

In [ ]:
plantas_energia.insert(0,'Id_Pais',plantas_energia['Pais'].map(dict(zip(paises.Nombre_Pais,paises.Id_Pais))))

In [ ]:
plantas_energia.drop(plantas_energia.columns[1:3], axis=1, inplace=True)

In [ ]:
database_username='root' # Nomre del cliente oculto (para WorkBench) que has creado
database_password='C13nc14d3d4t0s'
database_ip='localhost'
database_name='biodatascience'
database_conection=db.create_engine(f'mysql+pymysql://{database_username}:{database_password}@{database_ip}/{database_name}')
coneccion=database_conection.connect()
metadata=db.MetaData()

plantas_energia.to_sql('plantas_energia',coneccion, index=False) 
# data es  el nombre del Dataframe y ventas es el nombre que quiere para la tabla

8822